In [6]:
"""Code to format info on movies in a csv file for publication."""

# built-in libraries
import csv
from collections import OrderedDict, defaultdict
import datetime
from email.message import EmailMessage
import os.path
import re
import smtplib
import sqlite3
import sys

# installed with pip
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

# Create list of dictionaries for each row.
listings = []
scrapedir = '/Users/Doug/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-07-11.csv'
scrape_path = scrapedir + scrapefile
with open(scrape_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {
            'addy': row[0],
            'theater': row[1],
            'title': row[2],
            'mpaa': row[3],
            'stars': row[4],
            'times': row[5],
            'format': row[6],
            'synopsis': row[7],
            'date': row[8],
            'city': row[9]
        }
        listings.append(layout_dct)
        
del listings[0] #remove header row   
      
# Create Pandas dataframe for listings
df = pd.DataFrame(listings)
# Remove some duplicate listings
df = df[df.city != "Belleview, FL"]
df = df[df.city != "High Springs, FL"]
df = df[(df.city != "The Villages, FL") & (df.theater != "Belleview Cinemas")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
# fix up titles, times and days, keep just needed columns, sort rows by movie+day
df.title = df.title.apply(lambda x: '"' + str(x) + '"')
df.times = df.times.str.replace(':00', '')
df.times = df.times.str.replace('|', ',')
df.times = df.times.str.replace('p', ' p.m.')
df.times = df.times.str.replace('a', ' a.m.,')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df = df.drop('city', axis=1)
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop('date', axis=1)
df['screen'] = df['title'].str.cat(df['format'],sep=", ")
df.screen = df.screen.str.cat(df['mpaa'],sep=", ")
df = df[['theater', 'screen', 'movieday', 'times']]
df.sort_values(["screen", "movieday"], axis =0,
              ascending=True, inplace=True)
# Ocala market
oc6_df = df.loc[df['theater'] == 'Ocala Center 6']
ocd_dr = df.loc[df['theater'] == 'Ocala Drive-In']
bv_df = df.loc[df['theater'] == 'Belleview Cinemas']
mt_df = df.loc[df['theater'] == 'Marion Theatre']
omp_df = df.loc[df['theater'] == 'Old Mill Playhouse']
regaloca_df = df.loc[df['theater'] == 'Regal Hollywood Stadium 16 & IMAX - Ocala']
rialto_df = df.loc[df['theater'] == 'Rialto Theatre Spanish Springs Town Square']
amc_df = df.loc[df['theater'] == 'AMC Lake Square 12']
barn_df = df.loc[df['theater'] == 'Barnstorm Theater']

# Gainesville market
celeb_df = df.loc[df['theater'] == 'Celebration Pointe 10']
hipp_df = df.loc[df['theater'] == 'Hippodrome State Theatre']
butler_df = df.loc[df['theater'] == 'Regal Butler Town Center 14']
regalgnv_df = df.loc[df['theater'] == 'Regal Royal Park Stadium 16']
uac_df = df.loc[df['theater'] == 'UA Cinema 90 6']
starke_df = df.loc[df['theater'] == 'Florida Twin']


oc6_df = oc6_df.drop('theater', axis=1)
#oc6_df.set_index('screen')
#oc6_df.sort_values(["screen"], axis=0, 
#                 ascending=True, inplace=True)
oc6_df

,screen,movieday,times
441,"""Annabelle Comes Home"", Standard, R",Friday,"1 p.m. , 4 p.m. , 7 p.m. , 9:45 p.m."
20,"""Annabelle Comes Home"", Standard, R",Saturday,"1 p.m. , 4 p.m. , 7 p.m. , 9:45 p.m."
47,"""Annabelle Comes Home"", Standard, R",Sunday,"1 p.m. , 4 p.m. , 7 p.m. , 9:45 p.m."
428,"""Crawl (2019)"", Standard, R",Friday,"1 p.m. , 4 p.m. , 7 p.m. , 9:45 p.m."
46,"""Crawl (2019)"", Standard, R",Saturday,"1 p.m. , 4 p.m. , 7 p.m. , 9:45 p.m."
165,"""Crawl (2019)"", Standard, R",Sunday,"1 p.m. , 4 p.m. , 7 p.m. , 9:45 p.m."
470,"""Spider-Man: Far From Home (2019)"", Standard, ...",Friday,"1 p.m. , 4 p.m. , 7 p.m. , 9:45 p.m."
614,"""Spider-Man: Far From Home (2019)"", Standard, ...",Saturday,"1 p.m. , 4 p.m. , 7 p.m. , 9:45 p.m."
354,"""Spider-Man: Far From Home (2019)"", Standard, ...",Sunday,"1 p.m. , 4 p.m. , 7 p.m. , 9:45 p.m."
311,"""Stuber"", Standard, R",Friday,"1 p.m. , 4 p.m. , 7 p.m. , 9:45 p.m."


In [ ]:
oc6_df.to_csv('oc6.csv', index=False, header=False)

In [10]:
# Set index
#oc6_df.set_index(['screen', 'movieday'], inplace=True)
oc6_df.set_index(['screen', 'movieday'], inplace=True)
oc6

KeyError: 'screen'

In [ ]:
np.savetxt('oc6.txt', oc6_df.values, fmt='%s', delimiter=',', newline='\n', comments='#today')

In [ ]:
oc6_list = oc6_df.values.tolist()

In [ ]:
from itertools import groupby

for key, group in groupby(oc6_list, lambda x: x[0]):
    listOfThings = " and ".join([thing[1] for thing in group])
    print(key + ", " + listOfThings)

In [ ]:
i = 0
sizeofList = len(oc6_list) 
while i < sizeofList :
    if oc6_list[i][0] == oc6_list[i+1][0] and oc6_list[i][2] == oc6_list[i+1][2]:
        print(oc6_list[i][0] + ", " + oc6_list[i][1] + " at " + oc6_list[i][2]) 
    i += 1

In [ ]:
i = 0
sizeofList = len(oc6_list)
sizeofList = 6
while i < sizeofList :
    if oc6_list[i][1] == "Friday" and oc6_list[i+1][1] != "Saturday":
        print("It's a Friday only screening")
    elif oc6_list[i][1] == "Friday" and oc6_list[i+1][1] == "Saturday" and oc6_list[i+2][1] != "Sunday":
        print("It's a Friday-Saturday screening")
    elif oc6_list[i][1] == "Friday" and oc6_list[i][1] == "Saturday" and oc6_list[i][1] == "Sunday":
        print("We're here all weekend")
    elif oc6_list[i][1] == "Saturday" and oc6_list[i-1][1] != "Friday" and oc6_list[i+1]!= "Sunday":
        print("It's a Saturday only screening")
    elif oc6_list[i][1] == "Saturday" and oc6_list[i+1][1] != "Friday" and oc6_list[i+2][1] == "Sunday":
        print("It's a Saturday-Sunday screening")
    elif oc6_list[i][1] == "Saturday" and oc6_list[i-1][1] == "Friday" and oc6_list[i][1] == "Sunday":
        print("We're here all weekend, but we're saying that again. Shall we drop this line?")
    elif oc6_list[i][1] == "Sunday" and oc6_list[i-1][1] != "Saturday":
        print("It's a Sunday only screening")
    elif oc6_list[i][1] == "Sunday" and oc6_list[i-2][1] != "Friday" and oc6_list[i-1][1] == "Saturday":
        print("It's a Saturday-Sunday screening, but we already said that. Shall we drop this line?")
        print("We're here all weekend, but we're saying that again. Shall we drop this line?")
    else:
        print("This isn't working for some reason")
    
    i += 1

In [ ]:
i = 0
while i < 18:
    if oc6_list[i][1] == "Friday" and oc6_list[i+1][1] == "Saturday" and oc6_list[i+2][1] == "Sunday":
        print("We're here all weekend")
    elif oc6_list[i][1] == "Friday" and oc6_list[i+1][1] == "Saturday" and oc6_list[i+2][1] != "Sunday":
        print("Looks like a Friday-Saturday screening.")
    elif oc6_list[i][1] == "Friday" and oc6_list[i+1][1] != "Saturday":
        print("Looks like a Friday-only screening.")
    if oc6_list[i][1] == "Saturday" and oc6_list[i-1][1] == "Friday" and oc6_list[i+1][1] == "Sunday":
        print("We're here all weekend")
    elif oc6_list[i][1] == "Saturday" and oc6_list[i-1][1] == "Saturday" and oc6_list[i+1][1] != "Sunday":
        print("Looks like a Friday-Saturday screening, repeating.")
    elif oc6_list[i][1] == "Saturday" and oc6_list[i-1][1] != "Friday" and oc6_list[i][1] == "Sunday":
        print("Looks like a Saturday-Sunday screening.")
    elif oc6_list[i][1] == "Saturday" and oc6_list[i-1][1] != "Friday" and oc6_list[i+1][1] != "Sunday":
        print("Saturday only showing")
    else:
        print(i)
        
    i += 1

In [ ]:
i = 0
while i < 18:
    if oc6_list[i][] == "Friday" :
        fri = oc6_list[i][1]
    if oc6_list[i-1][1] == "Friday":
        lastfri = oc6_list[i-1][1]
    if oc6_list[i-1][1] == "Saturday":
        lastsat = oc6_list[i-1][1]
    if oc6_list[i-1][1] == "Sunday":
        lastsun = oc6_list[i-1][1]
    if oc6_list[i+1][1] == "Friday":
        nextfri = oc6_list[i+1][1]
    if oc6_list[i+1][1] == "Saturday":
        lastsat = oc6_list[i-1][1]
    if oc6_list[i+1][1] == "Sunday":
        lastsun = oc6_list[i-1][1]
    
    i += 1

In [ ]:
# KEEP THIS LOGIC

moviedict = {
    'Ocala6_AladinIMAX_Fri': {'times':'1, 2, 3'},
    'Ocala6_AladinIMAX_Sat': {'times':'1, 2, 3'},
    'Ocala6_AladinIMAX_Sun': {'times':'3, 2, 3'}
}
fri = str(moviedict['Ocala6_AladinIMAX_Fri']['times'])
sat = str(moviedict['Ocala6_AladinIMAX_Sat']['times'])
sun = str(moviedict['Ocala6_AladinIMAX_Sun']['times'])

if fri == sat and fri == sun: # want all common
    print("Friday-Sunday at " + fri) 
elif fri != sat and fri != sun and sat == sun: # want fri unique
    print("Friday at " + fri + ", Saturday-Sunday at " + sat)
elif sat != fri and sat != sun and fri == sun: # want sat unique
    print("Friday and Sunday at " + fri + ", Saturday at " + sat) 
elif sun != fri and sun != sat and fri == sat: # want sun unique
    print("Friday and Saturday at " + fri + ", Sunday at " + sun)
elif fri != sat and fri !=sun and sat !=sun: # want all unique
    print("Friday at " + fri + ", Saturday at " + sat + ",Sundaay at" + sun)
else:
    print("Show times didn't work out!")